In [7]:
from webutils import get_title_url_year
# 文章的google scholar 链接
url_scholar = 'https://scholar.google.com/scholar?cites=10525391200502590326&as_sdt=2005&sciodt=0,5&hl=zh-CN'
get_title_url_year(url_scholar)

正在抓取第 1 页...
10 10
正在抓取第 2 页...
10 10
正在抓取第 3 页...
10 10
正在抓取第 4 页...
10 10
正在抓取第 5 页...
10 10
正在抓取第 6 页...
10 10
正在抓取第 7 页...
10 10
正在抓取第 8 页...
10 10
正在抓取第 9 页...
10 10
正在抓取第 10 页...
10 10
正在抓取第 11 页...
10 10
正在抓取第 12 页...
10 10
正在抓取第 13 页...
10 10
正在抓取第 14 页...
10 10
正在抓取第 15 页...
10 10
正在抓取第 16 页...
10 10
正在抓取第 17 页...
10 10
正在抓取第 18 页...
10 10
正在抓取第 19 页...
10 10
正在抓取第 20 页...
10 10
条目抓取失败: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: MicrosoftEdge=133.0.3065.82); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF785D9E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF78602B1B4+2060276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF785F82338+1368440]
	(No symbol) [0x00007FF785B732A9]
	(No symbol) [0x00007FF785B7349B]
	(No symbol) [0x00007FF785

In [8]:
import pandas as pd
from utils import Paper

# 读取检索到的所有文章的title，url，year
df = pd.read_csv('data/all_articles.csv')
papers = []
for index, row in df.iterrows():
    title = row[0]
    url = row[1]
    year = row[2]
    papers.append(Paper(title, url, year=year))


In [9]:
from webutils import arxiv,nips,acl,openreview,ieee
import csv

arxiv([p for p in papers if 'arxiv' in p.url])
nips([p for p in papers if 'neurips' in p.url])
acl([p for p in papers if 'aclanthology' in p.url])
openreview([p for p in papers if 'openreview' in p.url])
ieee([p for p in papers if 'ieee.org' in p.url])

seen = set()
with open("data/papers_with_author.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "year"])
    keywords = {'arxiv', 'neurips', 'aclanthology', 'openreview', 'ieee.org'}
    for paper in (p for p in papers if any(keyword in p.url for keyword in keywords)):
        author_list = []
        for author in paper.authors:
            if author not in seen:
                seen.add(author)
                author_list.append(author)
        writer.writerow([paper.title, paper.url, author_list, paper.year])
        # 保存'arxiv', 'neurips', 'aclanthology', 'openreview', 'ieee.org' 的相关信息,其他出处文章请自行查找并加入

['Jonathan Svirsky', 'Ofir Lindenbaum']
['Quanming Yao', 'Yongqi Zhang', 'Yaqing Wang', 'Nan Yin', 'James Kwok', 'Qiang Yang']
['Kexuan Li']
['Kristof Lommers', 'Ouns El Harzli', 'Jack Kim']
['Camilo Ruiz', 'Hongyu Ren', 'Kexin Huang', 'Jure Leskovec']
['Sarthak Jain', 'Sandra E. Safo']
['Thomas Corcoran', 'Rafael Zamora-Resendiz', 'Xinlian Liu', 'Silvia Crivelli']
['Liu Yang', 'Shuowei Cai', 'Di Chai', 'Junxue Zhang', 'Han Tian', 'Yilun Jin', 'Kun Guo', 'Kai Chen', 'Qiang Yang']
['Andrei Margeloiu', 'Nikola Simidjievski', 'Pietro Lio', 'Mateja Jamnik']
['Bin Luo', 'Susan Halabi']
['Yinchen Shen', 'Zhiguo Wang', 'Ruoyu Sun', 'Xiaojing Shen']
['J. Mangos', 'N. Birbilis']
['Atsutoshi Kumagai', 'Tomoharu Iwata', 'Yasuhiro Fujiwara']
['Pin Ji', 'Yang Feng', 'Weitao Huang', 'Jia Liu', 'Zhihong Zhao']
['Yue Mao', 'Yi Shen', 'Gang Qin', 'Longjun Cai']
['Kexuan Li']
['Amit Mishra', 'Pranath Reddy', 'Rahul Nigam']
['Zhiyuan Liu', 'Jörn Schulz', 'Mohsen Taheri', 'Martin Styner', 'James Damon', '

In [10]:
import ast
from homepage_utils import get_author_homepage

df = pd.read_csv('data/papers_with_author.csv')
papers = []
print(len(df))
for index, row in df.iterrows():
    title = row[0]
    url = row[1]
    str = row[2]
    year = row[3]
    author_list = ast.literal_eval(str)
    print(author_list)
    papers.append(Paper(title, url, authors=author_list, year=year))
for paper in papers:
    for author in paper.authors:
        homepage = get_author_homepage(author,api_key="e51c6f1cd36f593862288e55738290a6cd778ed1")
        print(homepage)
        paper.author_homepages.append(homepage)
with open("data/papers_with_homepage.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "Authors-HomePage", "Year-Source"])
    for paper in [p for p in papers]:
        writer.writerow([paper.title, paper.url, paper.authors, paper.author_homepages, paper.year])

47
['Camilo Ruiz', ' Hongyu Ren', ' Kexin Huang', ' Jure Leskovec']
['Xiaofeng Zhang', 'Zhangyang Wang', 'Dong Liu', 'Qing Ling']
['Debasmit Das', 'Sungrack Yun', 'Fatih Porikli']
['Qifeng Lin']
['Dinesh Singh', 'Hector Climente-Gonzalez', 'Mathis Petrovich', 'Eiryo Kawakami', 'Makoto Yamada']
['Jonathan Svirsky', 'Ofir Lindenbaum']
['Wasu Kudisthalert', 'Kitsuchart Pasupa', 'Sissades Tongsima']
['Atsutoshi Kumagai', ' Tomoharu Iwata', ' Yasutoshi Ida', ' Yasuhiro Fujiwara']
['Ashis Kumar Mandal', 'MD. Nadim', 'Hasi Saha', 'Tangina Sultana', 'Md. Delowar Hossain', 'Eui-Nam Huh']
['Sheetal Rajpal', 'Manoj Agarwal', 'Virendra Kumar', 'Anamika Gupta', 'Naveen Kumar']
['Quanming Yao', 'Yongqi Zhang', 'Yaqing Wang', 'Nan Yin', 'James Kwok', 'Qiang Yang']
['Kexuan Li']
['Lechuan Li', 'Chonghao Zhang', 'Shiyu Liu', 'Hannah Guan', 'Yu Zhang']
['Tingyang Wei', 'Wei-Li Liu', 'Jinghui Zhong', 'Yue-Jiao Gong']
['Shanta Chowdhury', 'Xishuang Dong', 'Xiangfang Li']
['Mohammad Sultan Mahmud', 'Xiangh

In [ ]:
from find_citation import extract_text_with_fitz,find_citation_symbol,find_content
from utils import read_data
import os
import csv
keywords = {
            'Yu et al., 2023','Yu et al.,2023','Yu et al.,\n2023','Yu et al.,\n 2023','Yu et al., \n2023', 'Yu et al., 2024','Yu et al.,2024','Yu et al.,\n2024','Yu et al.,\n 2024','Yu et al., \n2024',
            'Yu et al., (2023)','Yu et al.,(2023)','Yu et al.,\n(2023)','Yu et al.,\n (2023)','Yu et al., \n(2023)', 'Yu et al., (2024)','Yu et al.,(2024)','Yu et al.,\n(2024)','Yu et al.,\n (2024)','Yu et al., \n(2024)',
            'Yu et al. 2023','Yu et al.2023','Yu et al.\n2023','Yu et al.\n 2023','Yu et al. \n2023', 'Yu et al. 2024','Yu et al.2024','Yu et al.\n2024','Yu et al.\n 2024','Yu et al. \n2024',
            'Yu et al. (2023)','Yu et al.(2023)','Yu et al.\n(2023)','Yu et al.\n (2023)','Yu et al. \n(2023)', 'Yu et al. (2024)','Yu et al.(2024)','Yu et al.\n(2024)','Yu et al.\n (2024)','Yu et al. \n(2024)',             
            }

papers = read_data('./data/combined_result.csv')
os.makedirs('./downloads',exist_ok=True)

for paper in papers:
    if 'arxiv' not in paper.url:
        continue
    arxiv_id = paper.url.split('/')[-1]
    arxiv_pdf = os.path.join('downloads',f'{arxiv_id}.pdf')
    text = extract_text_with_fitz(arxiv_pdf).replace('\n','')
    content = find_content(text,keywords)
    symbol = find_citation_symbol(content,keywords)
    paper.set_content(content)
    paper.set_symbol(symbol)

with open("data/papers_with_content.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "Authors-HomePage", "Year-Source", "Content", "Symbol"])
    for paper in [p for p in papers]:
        writer.writerow([paper.title, paper.url, paper.authors, paper.author_homepages, paper.year, paper.content, paper.symbol])


In [ ]:
from pptutils import generate
from utils import read_data


papers = read_data('./data/papers_with_homepage.csv')
generate(papers,'./data/test.pptx')